# Converting master gdf to Master Intersection Collisions & Attributes

The following code takes in the master dataframe and performs aa function that prints out a CSV which contains
- Intersections
- Total number of collisions at intersections across all time
- Average traffic volume on a daily basis.
- Columns based on the intersection data.
- R value based on theory.

#### Import packages and libraries

In [1]:
import pandas as pd
import seaborn as sns
from datetime import datetime
import numpy as np
import requests

# 1.1 Import Master Dataframe

In [2]:
master_df = pd.read_csv('master_gdf.csv')
master_df.head()

C:\Users\ianto\Python\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,47,49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_ex_peds,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other
0,1237939,2011.0,2011-03-09,726,2011-03-09 07:26:00,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,208.0,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0
1,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
2,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
3,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
4,1222866,2011.0,2011-03-12,1917,2011-03-12 19:17:00,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,222.0,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0


In [3]:
master_df['count_location']

0       BATHURST ST AT COLLEGE ST (PX 300)
1          JANE ST AT ST JOHNS RD (PX 523)
2          JANE ST AT ST JOHNS RD (PX 523)
3          JANE ST AT ST JOHNS RD (PX 523)
4           CHURCH ST AT GOULD ST (PX 993)
                       ...                
8212                 AVA RD AT WINNETT AVE
8213                 AVA RD AT WINNETT AVE
8214                 AVA RD AT WINNETT AVE
8215    BLOOR ST AT MONTROSE AVE (PX 1928)
8216    BLOOR ST AT MONTROSE AVE (PX 1928)
Name: count_location, Length: 8217, dtype: object

# 2.1 Group Intersections together and count number of collisions for each
This is done across the time period of interest.

In [4]:
intersections = master_df.groupby('count_location')['count_datetime'].count()
intersections

count_location
# 1645 SHEPPARD AVE AT DOWNSVIEW DELL PARK / HINCKS DELLCREST CTR     2
# 3030 BIRCHMOUNT RD AT BIRCHMOUNT RD (NORTH DRIVEWAY)                3
# 4400 DUFFERIN ST AT DUFFERIN ST & 250M N STANSTEAD DR (PX 1625)     3
1630 LAWRENCE AVE W AT 55M E OF OLYMPIA DR (PX 1624)                  2
3410 MIDLAND AVE                                                     16
                                                                     ..
YONGE ST AT ORCHARD VIEW BLVD & ROEHAMPTON AVE (PX 2144)              6
YONGE ST AT SHAFTESBURY AVE                                           4
YONGE ST AT WILSON AVE & YORK MILLS RD                               19
YORK MILLS RD AT SILVERDALE CRES & VALENTINE DR                       2
YORK ST AT GARDINER EXPY RAMP                                         2
Name: count_datetime, Length: 1701, dtype: int64

In [5]:
intersections = intersections.to_frame()

In [6]:
# Set up dataset to have the appropriate dataframe format for merging.
intersections = intersections.reset_index()
intersections = intersections.rename(columns = {'count_location' : 'intersections', 
                                                'count_datetime' : 'num_of_collisions'}
                                    )
intersections = intersections.set_index('intersections')
intersections.head()

,num_of_collisions
intersections,
# 1645 SHEPPARD AVE AT DOWNSVIEW DELL PARK / HINCKS DELLCREST CTR,2
# 3030 BIRCHMOUNT RD AT BIRCHMOUNT RD (NORTH DRIVEWAY),3
# 4400 DUFFERIN ST AT DUFFERIN ST & 250M N STANSTEAD DR (PX 1625),3
1630 LAWRENCE AVE W AT 55M E OF OLYMPIA DR (PX 1624),2
3410 MIDLAND AVE,16


# 2.2 Create dataframe to take the sum and mean of traffic per intersection
The following calcualtes the total sum of the traffic an intersection has experienced for the duration of the analysis period.

In [7]:
inter_traffic_data = master_df.iloc[:, np.r_[54, 56:104]]
inter_traffic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8217 entries, 0 to 8216
Data columns (total 49 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   count_location    8217 non-null   object 
 1   daily_sb_cars_r   8217 non-null   float64
 2   daily_sb_cars_t   8217 non-null   float64
 3   daily_sb_cars_l   8217 non-null   float64
 4   daily_nb_cars_r   8217 non-null   float64
 5   daily_nb_cars_t   8217 non-null   float64
 6   daily_nb_cars_l   8217 non-null   float64
 7   daily_wb_cars_r   8217 non-null   float64
 8   daily_wb_cars_t   8217 non-null   float64
 9   daily_wb_cars_l   8217 non-null   float64
 10  daily_eb_cars_r   8217 non-null   float64
 11  daily_eb_cars_t   8217 non-null   float64
 12  daily_eb_cars_l   8217 non-null   float64
 13  daily_sb_truck_r  8217 non-null   float64
 14  daily_sb_truck_t  8217 non-null   float64
 15  daily_sb_truck_l  8217 non-null   float64
 16  daily_nb_truck_r  8217 non-null   float64


In [8]:
# Set up dataframe for calculation such that it only has the traffic of interest.
inter_traffic_data = inter_traffic_data.set_index('count_location')
inter_traffic_data = inter_traffic_data.drop(columns = ['daily_sx_peds', 
                                                        'daily_nx_peds', 
                                                        'daily_wx_peds', 
                                                        'daily_ex_peds']
                                            )
inter_traffic_data.head()

,daily_sb_cars_r,daily_sb_cars_t,daily_sb_cars_l,daily_nb_cars_r,daily_nb_cars_t,daily_nb_cars_l,daily_wb_cars_r,daily_wb_cars_t,daily_wb_cars_l,daily_eb_cars_r,...,daily_eb_bus_t,daily_eb_bus_l,daily_nx_bike,daily_sx_bike,daily_ex_bike,daily_wx_bike,daily_nx_other,daily_sx_other,daily_ex_other,daily_wx_other
count_location,,,,,,,,,,,,,,,,,,,,,
BATHURST ST AT COLLEGE ST (PX 300),417.0,3954.0,1190.0,929.0,3690.0,194.0,1078.0,3326.0,142.0,677.0,...,183.0,19.0,253.0,234.0,797.0,1647.0,0.0,0.0,0.0,0.0
JANE ST AT ST JOHNS RD (PX 523),92.0,4397.0,397.0,172.0,4413.0,79.0,475.0,247.0,201.0,154.0,...,2.0,5.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0
JANE ST AT ST JOHNS RD (PX 523),92.0,4397.0,397.0,172.0,4413.0,79.0,475.0,247.0,201.0,154.0,...,2.0,5.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0
JANE ST AT ST JOHNS RD (PX 523),92.0,4397.0,397.0,172.0,4413.0,79.0,475.0,247.0,201.0,154.0,...,2.0,5.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0
CHURCH ST AT GOULD ST (PX 993),0.0,4023.0,77.0,104.0,3132.0,0.0,166.0,0.0,119.0,85.0,...,0.0,0.0,567.0,387.0,129.0,156.0,0.0,0.0,0.0,0.0


In [9]:
# Take total sum of traffic
inter_traffic_data['total_traffic'] = inter_traffic_data.sum(axis = 1)
inter_traffic_data.head()

,daily_sb_cars_r,daily_sb_cars_t,daily_sb_cars_l,daily_nb_cars_r,daily_nb_cars_t,daily_nb_cars_l,daily_wb_cars_r,daily_wb_cars_t,daily_wb_cars_l,daily_eb_cars_r,...,daily_eb_bus_l,daily_nx_bike,daily_sx_bike,daily_ex_bike,daily_wx_bike,daily_nx_other,daily_sx_other,daily_ex_other,daily_wx_other,total_traffic
count_location,,,,,,,,,,,,,,,,,,,,,
BATHURST ST AT COLLEGE ST (PX 300),417.0,3954.0,1190.0,929.0,3690.0,194.0,1078.0,3326.0,142.0,677.0,...,19.0,253.0,234.0,797.0,1647.0,0.0,0.0,0.0,0.0,23925.0
JANE ST AT ST JOHNS RD (PX 523),92.0,4397.0,397.0,172.0,4413.0,79.0,475.0,247.0,201.0,154.0,...,5.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0,11620.0
JANE ST AT ST JOHNS RD (PX 523),92.0,4397.0,397.0,172.0,4413.0,79.0,475.0,247.0,201.0,154.0,...,5.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0,11620.0
JANE ST AT ST JOHNS RD (PX 523),92.0,4397.0,397.0,172.0,4413.0,79.0,475.0,247.0,201.0,154.0,...,5.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0,11620.0
CHURCH ST AT GOULD ST (PX 993),0.0,4023.0,77.0,104.0,3132.0,0.0,166.0,0.0,119.0,85.0,...,0.0,567.0,387.0,129.0,156.0,0.0,0.0,0.0,0.0,9440.0


In [10]:
# Get average traffic flow on a daily basis.
inter_traffic_data.reset_index()
inter_traffic_avg = inter_traffic_data.groupby('count_location')['total_traffic'].mean()
inter_traffic_avg = inter_traffic_avg.to_frame()
inter_traffic_avg.head()

,total_traffic
count_location,
# 1645 SHEPPARD AVE AT DOWNSVIEW DELL PARK / HINCKS DELLCREST CTR,15613.0
# 3030 BIRCHMOUNT RD AT BIRCHMOUNT RD (NORTH DRIVEWAY),10644.0
# 4400 DUFFERIN ST AT DUFFERIN ST & 250M N STANSTEAD DR (PX 1625),42055.0
1630 LAWRENCE AVE W AT 55M E OF OLYMPIA DR (PX 1624),18320.0
3410 MIDLAND AVE,12766.0


# 3.1 Merge datasets

In [11]:
merged_intersections = pd.merge(intersections, inter_traffic_avg, left_index=True, right_index=True)
merged_intersections.head()

,num_of_collisions,total_traffic
intersections,,
# 1645 SHEPPARD AVE AT DOWNSVIEW DELL PARK / HINCKS DELLCREST CTR,2,15613.0
# 3030 BIRCHMOUNT RD AT BIRCHMOUNT RD (NORTH DRIVEWAY),3,10644.0
# 4400 DUFFERIN ST AT DUFFERIN ST & 250M N STANSTEAD DR (PX 1625),3,42055.0
1630 LAWRENCE AVE W AT 55M E OF OLYMPIA DR (PX 1624),2,18320.0
3410 MIDLAND AVE,16,12766.0


# 3.2 Calcualte R with merged datasets

In [12]:
# Create columns with assumptions for to calculate R. (Note: assume average interesection width is 5 lanes)
merged_intersections['average_int_width'] = 5 * 3 / 1000
merged_intersections['years'] = 11
merged_intersections.head()

,num_of_collisions,total_traffic,average_int_width,years
intersections,,,,
# 1645 SHEPPARD AVE AT DOWNSVIEW DELL PARK / HINCKS DELLCREST CTR,2,15613.0,0.015,11
# 3030 BIRCHMOUNT RD AT BIRCHMOUNT RD (NORTH DRIVEWAY),3,10644.0,0.015,11
# 4400 DUFFERIN ST AT DUFFERIN ST & 250M N STANSTEAD DR (PX 1625),3,42055.0,0.015,11
1630 LAWRENCE AVE W AT 55M E OF OLYMPIA DR (PX 1624),2,18320.0,0.015,11
3410 MIDLAND AVE,16,12766.0,0.015,11


In [13]:
# Calculate R in accordance to equation
merged_intersections['r'] = (merged_intersections['num_of_collisions'] * 100000 / 
                             (365 * 
                              merged_intersections['total_traffic'] * 
                              merged_intersections['average_int_width'] * 
                              merged_intersections['years'])
                            )  
merged_intersections.head()

,num_of_collisions,total_traffic,average_int_width,years,r
intersections,,,,,
# 1645 SHEPPARD AVE AT DOWNSVIEW DELL PARK / HINCKS DELLCREST CTR,2,15613.0,0.015,11,0.212700
# 3030 BIRCHMOUNT RD AT BIRCHMOUNT RD (NORTH DRIVEWAY),3,10644.0,0.015,11,0.467993
# 4400 DUFFERIN ST AT DUFFERIN ST & 250M N STANSTEAD DR (PX 1625),3,42055.0,0.015,11,0.118448
1630 LAWRENCE AVE W AT 55M E OF OLYMPIA DR (PX 1624),2,18320.0,0.015,11,0.181271
3410 MIDLAND AVE,16,12766.0,0.015,11,2.081078


# 4.1 Merge Intersection Attributes with R

In [14]:
# create copy of master_df without more than one intersection in intersection column
inter_dupli_dropped = master_df.drop_duplicates(subset = 'count_location')
inter_dupli_dropped.head()

,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_ex_peds,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other
0,1237939,2011.0,2011-03-09,726,2011-03-09 07:26:00,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,208.0,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0
1,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
4,1222866,2011.0,2011-03-12,1917,2011-03-12 19:17:00,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,222.0,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0
6,1232760,2011.0,2011-03-14,1307,2011-03-14 13:07:00,CHURCH ST,CHARLES ST E,None,Minor Arterial,Toronto and East York,...,127.0,111.0,38.0,44.0,21.0,8.0,0.0,0.0,0.0,0.0
8,1233009,2011.0,2011-03-15,852,2011-03-15 08:52:00,KING Stre E,PRINCESS Stre,None,Major Arterial,Toronto and East York,...,12.0,22.0,4.0,15.0,91.0,9.0,0.0,0.0,45.0,37.0


In [15]:
inter_dupli_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1701 entries, 0 to 8215
Columns: 152 entries, ACCNUM to hourly_wx_other
dtypes: float64(98), int64(1), object(53)
memory usage: 2.0+ MB


In [16]:
# Create dataframe with just the intersection columns and the intersection attributes
inter_dupli_dropped = inter_dupli_dropped[['count_location', 'ROAD_CLASS', 'TRAFFCTL']]
inter_dupli_dropped.head()

,count_location,ROAD_CLASS,TRAFFCTL
0,BATHURST ST AT COLLEGE ST (PX 300),Major Arterial,No Control
1,JANE ST AT ST JOHNS RD (PX 523),Major Arterial,Traffic Signal
4,CHURCH ST AT GOULD ST (PX 993),Minor Arterial,Traffic Signal
6,CHARLES ST E AT CHURCH ST (PX 225),Minor Arterial,No Control
8,KING ST E AT PRINCESS ST,Major Arterial,No Control


In [17]:
# Perform merge
intersection_r = pd.merge(inter_dupli_dropped, merged_intersections, left_on = 'count_location', right_index = True)
intersection_r.head()

,count_location,ROAD_CLASS,TRAFFCTL,num_of_collisions,total_traffic,average_int_width,years,r
0,BATHURST ST AT COLLEGE ST (PX 300),Major Arterial,No Control,18,20083.944444,0.015,11,1.488150
1,JANE ST AT ST JOHNS RD (PX 523),Major Arterial,Traffic Signal,3,11620.000000,0.015,11,0.428685
4,CHURCH ST AT GOULD ST (PX 993),Minor Arterial,Traffic Signal,10,9440.000000,0.015,11,1.758941
6,CHARLES ST E AT CHURCH ST (PX 225),Minor Arterial,No Control,2,11585.000000,0.015,11,0.286653
8,KING ST E AT PRINCESS ST,Major Arterial,No Control,11,5304.000000,0.015,11,3.443597


In [18]:
intersection_r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1701 entries, 0 to 8215
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   count_location     1701 non-null   object 
 1   ROAD_CLASS         1701 non-null   object 
 2   TRAFFCTL           1701 non-null   object 
 3   num_of_collisions  1701 non-null   int64  
 4   total_traffic      1701 non-null   float64
 5   average_int_width  1701 non-null   float64
 6   years              1701 non-null   int64  
 7   r                  1701 non-null   float64
dtypes: float64(3), int64(2), object(3)
memory usage: 119.6+ KB


In [19]:
# Save New df as CSV.
intersection_r.to_csv("intersection_r.csv", index=False)

inter_att = (requests.get('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c83f641-7808-49ba-b80f-7011851d4e27/resource/8e825e33-d7e1-4e59-b247-5868bf7d66a9/download/Centreline%20Intersection.geojson').
            json()
                   )

inter_att

inter_attri = master_df.iloc[:, np.r_[54:152]]
inter_attri.info()